In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

pd.set_option('display.max_columns', None)


In [2]:
## fantrax rosters

fx = r"C:\Users\mikej\Desktop\fantrax_golf\fx_wk9.csv"

teams = pd.read_csv(fx ,usecols=['Player','Status','Roster Status'])
teams.columns = ['player','team','active_reserve']
teams_dict = {'919':'Philly919','u_c':'unit_circle','NT 8':'Sneads Foot','NT 4':'New Team 4','txms':'txmoonshine','MG':'Team Gamble','grrr':'Putt Pirates','[AW]':'AlphaWired'}
teams['team'] = teams.team.map(teams_dict)
teams = teams.loc[teams.active_reserve=='Active'].set_index('player')

In [26]:
## live scoring feed

file = "https://feeds.datagolf.com/preds/live-tournament-stats?stats=sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_bs,sg_total,distance,accuracy,gir,prox_fw,prox_rgh,scrambling&round=event_avg&display=value&file_format=csv&key=e297e933c3ad47d71ec1626c299e"

live = round(pd.read_csv(file),2).rename(columns={'player_name':'player'})#.set_index('player')

names = live['player'].str.split(expand=True)
names[0] = names[0].str.rstrip(",")
names[1] = names[1].str.rstrip(",")
names['player'] = names[1] + " " + names[0]
names['player'] = np.where(names['player']=='Matt Fitzpatrick', 'Matthew Fitzpatrick', names['player'])
names['player'] = np.where(names['player']=='Si Kim', 'Si Woo Kim', names['player'])
names['player'] = np.where(names['player']=='Min Lee', 'Min Woo Lee', names['player'])
names['player'] = np.where(names['player']=='Byeong An', 'Byeong Hun An', names['player'])
names['player'] = np.where(names['player']=='Rooyen Van', 'Erik Van Rooyen', names['player'])
live = live.set_index(names.player)
live

,event_name,last_updated,stat_display,position,player,dg_id,stat_round,total,round,thru,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_bs,sg_total,distance,accuracy,gir,prox_fw,prox_rgh,scrambling
player,,,,,,,,,,,,,,,,,,,,,,,
Luke List,Arnold Palmer Invitational presented by Master...,2024-03-07 15:39:08 UTC,values,1,"List, Luke",7960,event_avg,-4,-4.0,11.0,1.54,0.98,-0.79,1.28,1.47,0.49,3.00,292.3,0.78,0.73,38.27,84.22,0.75
Justin Lower,NaN,NaN,NaN,T2,"Lower, Justin",17723,event_avg,-3,-3.0,11.0,1.90,0.35,0.67,-0.92,0.10,-0.25,2.00,276.0,0.44,0.45,25.65,61.77,0.80
Hideki Matsuyama,NaN,NaN,NaN,T2,"Matsuyama, Hideki",13562,event_avg,-3,-3.0,5.0,1.07,-0.17,2.04,0.01,1.88,2.05,2.96,280.7,0.75,0.80,13.74,0.00,1.00
Chris Kirk,NaN,NaN,NaN,T2,"Kirk, Chris",12423,event_avg,-3,-3.0,4.0,1.22,0.24,0.75,0.70,1.68,1.45,2.91,299.0,1.00,1.00,15.50,0.00,1.00
Min Woo Lee,NaN,NaN,NaN,T5,"Lee, Min Woo",16841,event_avg,-2,-2.0,10.0,1.07,-0.04,-0.58,0.55,-0.06,-0.02,1.00,281.4,0.75,0.70,31.89,0.00,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Rory McIlroy,NaN,NaN,NaN,WAITING,"McIlroy, Rory",10091,event_avg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ludvig Aberg,NaN,NaN,NaN,WAITING,"Aberg, Ludvig",23950,event_avg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Shane Lowry,NaN,NaN,NaN,WAITING,"Lowry, Shane",13900,event_avg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
team_color={
                "Philly919": 'rgb(127,60,141)',
                "unit_cirle": 'rgb(17,165,121)',
                "AlphaWired": 'rgb(57,105,172)',
                "Sneads Foot": 'rgb(242,183,1)',
                "New Team 4": 'rgb(231,63,116)',
                "Team Gamble": 'rgb(230,131,16)',
                "txmoonshine": 'rgb(0,134,139)',
                "Putt Pirates": 'rgb(165,170,153)'}

In [8]:
live_merged = pd.merge(teams, live, how='left', left_index=True, right_index=True)[['player','team','position','total','round','thru','sg_putt','sg_t2g','sg_total','gir']].fillna(0).sort_values('total')
live_merged['holes_remaining'] = (72 - (live_merged['thru']).fillna(0))
live_merged['holes_remaining'] = np.where(live_merged['position']=='CUT',0,live_merged['holes_remaining']).astype('int')
live_merged = live_merged.reset_index(drop=True)
live_merged

,player,team,position,total,round,thru,sg_putt,sg_t2g,sg_total,gir,holes_remaining
0,0,New Team 4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72
1,0,Team Gamble,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72
2,0,txmoonshine,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72
3,0,Putt Pirates,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72
4,0,Philly919,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72
5,0,New Team 4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72
6,0,Philly919,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72
7,0,AlphaWired,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72
8,0,unit_circle,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72
9,0,Sneads Foot,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72


In [94]:
# make leaderboard

live_leaderboard = live_merged[['player','team','position','total','round','thru']].fillna(0).sort_values('total')

live_leaderboard[['total','round','thru']] = live_leaderboard[['total','round','thru']].astype('int')

# live_leaderboard = live_leaderboard.rename(columns={'player':'Player','team':'Team','position':'Pos','total':'Total','round':'Round','thru':'Thru'})

def highlight_rows(row):
    value = row.loc['Team']
    if value == 'unit_circle':
        color = '#FF99FF' # Pink
    elif value == 'Philly919':
        color = '#7f3c8d' # Purple
    elif value == 'AlphaWired':
        color = '#3969ac' # Blue
    elif value == 'Sneads Foot':
        color = '#f2b701' # Gold
    elif value == 'New Team 4':
        color = '#e73f74' # Magenta
    elif value == 'Team Gamble':
        color = '#e68310' # Orange
    elif value == 'txmoonshine':
        color = '#00868b' # Aqua
    else:
        color = '#a5aa99' # Grey
    return ['background-color: {}'.format(color) for r in row]

live_leaderboard.rename(columns={'player':'Player','team':'Team','position':'Pos','total':'Total','round':'Round','thru':'Thru'}).style.apply(highlight_rows, axis=1)

,Player,Team,Pos,Total,Round,Thru
0,"Lee, Min Woo",unit_circle,T2,-14,-4,18
1,"Van Rooyen, Erik",New Team 4,T2,-14,-8,18
2,"Knapp, Jake",unit_circle,T4,-13,-5,18
3,"Lowry, Shane",unit_circle,T4,-13,0,18
4,"Young, Cameron",New Team 4,T4,-13,-5,18
5,"Ryder, Sam",Sneads Foot,T21,-10,-5,18
6,"McIlroy, Rory",Sneads Foot,T21,-10,-3,18
7,"An, Byeong Hun",Team Gamble,T21,-10,-6,18
8,"Hoge, Tom",Team Gamble,T28,-9,-3,18
9,"Pavon, Matthieu",Team Gamble,T28,-9,-1,18


In [102]:
live_phr = live_merged.groupby('team')[['holes_remaining']].sum().reset_index().rename(columns={'team':'Team','holes_remaining':'PHR','total':'To Par'})
live_phr.style.apply(highlight_rows, axis=1)

,Team,PHR
0,AlphaWired,432
1,New Team 4,162
2,Philly919,162
3,Putt Pirates,360
4,Sneads Foot,270
5,Team Gamble,216
6,txmoonshine,396
7,unit_circle,360


In [120]:
# Team Score bar

thru_cut_df = live_leaderboard.groupby('team')['total'].sum().sort_values()#.rename(columns={'team':'thruCut'})

thru_cut_bar = px.bar(thru_cut_df,
                     template='presentation',
                     labels={'value':'','team':''},
                     text_auto=True,
                     height=250,
                     title='Team Score to Par')

thru_cut_bar.update_layout(showlegend=False,title_x=.25)
thru_cut_bar.update_yaxes(showticklabels=False,showgrid=False)
thru_cut_bar.update_traces(marker_color='rgb(200,200,200)',marker_line_width=1.5, opacity=0.6)

thru_cut_bar.show()

In [108]:
# Thru Cut Bar

thru_cut_df = live_leaderboard[live_leaderboard.position!='CUT']['team'].value_counts()#.rename(columns={'team':'thruCut'})

thru_cut_bar = px.bar(thru_cut_df,
                 template='presentation',
                 labels={'value':'','index':''},
                 text_auto=True,
                 height=250,
                 log_y=True,
                 title='Players Thru the Cut')

thru_cut_bar.update_layout(showlegend=False,title_x=.25)
thru_cut_bar.update_yaxes(showticklabels=False,showgrid=False)
thru_cut_bar.update_traces(marker_color='rgb(200,200,200)',marker_line_width=1.5, opacity=0.6)

In [121]:
print(f"unit_circle has {live_merged[live_merged.team=='unit_circle']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='unit_circle'].drop(columns=['holes_remaining','team'])

unit_circle has 360 Holes Remaining


,player,position,total,round,thru,sg_putt,sg_t2g,sg_total,gir
0,"Lee, Min Woo",T2,-14.0,-4.0,18.0,0.24,2.17,2.42,0.64
2,"Knapp, Jake",T4,-13.0,-5.0,18.0,0.37,1.80,2.16,0.71
3,"Lowry, Shane",T4,-13.0,0.0,18.0,0.83,1.34,2.16,0.81
12,"Conners, Corey",T41,-7.0,-5.0,18.0,-0.61,1.27,0.66,0.76
29,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
36,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00


In [ ]:
# live look at all 48 active players with player as index

print(f"Philly919 has {live_merged[live_merged.team=='Philly919']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='Philly919'].drop(columns=['holes_remaining','team'])

In [ ]:
# live look at all 48 active players with player as index

print(f"Team Gamble has {live_merged[live_merged.team=='Team Gamble']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='Team Gamble'].drop(columns=['holes_remaining','team'])

In [ ]:
# live look at all 48 active players with player as index

print(f"New Team 4 has {live_merged[live_merged.team=='New Team 4']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='New Team 4'].drop(columns=['holes_remaining','team'])

In [158]:
unit_circle = teams.loc[(teams.team=='unit_circle') & (teams.active_reserve=='Active')].index

live['thru_hp'] = live['thru'] + 18

# live[live.player.isin([unit_circle])]#[['player','total','round','thru','thru_hp']].reset_index(drop=True)

Index(['Patrick Cantlay', 'Tommy Fleetwood', 'Min Woo Lee', 'Corey Conners',
       'Shane Lowry', 'Jake Knapp'],
      dtype='object', name='player')

In [172]:
def get_active_rosters(teams):
    active_rosters = []
    
    for team in teams.team.unique():
        one_roster = teams.loc[(teams.team==team) & (teams.active_reserve=='Active')].index
        active_rosters.append(one_roster)
        
    cols = teams.team.unique()
    active_rosters = pd.DataFrame(active_rosters, index=cols).T
    return active_rosters

In [173]:
get_active_rosters(teams)

,AlphaWired,unit_circle,Team Gamble,txmoonshine,Philly919,Sneads Foot,Putt Pirates,New Team 4
0,Sam Burns,Patrick Cantlay,Eric Cole,Tom Kim,Luke List,Rory McIlroy,Scottie Scheffler,Cameron Young
1,Xander Schauffele,Tommy Fleetwood,Byeong Hun An,Collin Morikawa,Sungjae Im,Rickie Fowler,Sahith Theegala,Erik Van Rooyen
2,Jordan Spieth,Min Woo Lee,Stephan Jaeger,J.T. Poston,Russell Henley,Sam Ryder,Will Zalatoris,Akshay Bhatia
3,Jason Day,Corey Conners,Matthieu Pavon,Matthew Fitzpatrick,Chris Kirk,Cameron Champ,Emiliano Grillo,Justin Suh
4,Brian Harman,Shane Lowry,Tom Hoge,Keegan Bradley,Justin Rose,Nick Dunlap,Christiaan Bezuidenhout,Camilo Villegas
5,Harris English,Jake Knapp,Chesson Hadley,Nicolai Hojgaard,Daniel Berger,Parker Coody,Kurt Kitayama,Thorbjorn Olesen


In [163]:
active_rosters = []

for team in teams.team.unique():
    one_roster = teams.loc[(teams.team==team) & (teams.active_reserve=='Active')].index
    active_rosters.append(one_roster)

cols = teams.team.unique()
active_rosters = pd.DataFrame(active_rosters, index=cols).T
active_rosters

,AlphaWired,unit_circle,Team Gamble,txmoonshine,Philly919,Sneads Foot,Putt Pirates,New Team 4
0,Sam Burns,Patrick Cantlay,Eric Cole,Tom Kim,Luke List,Rory McIlroy,Scottie Scheffler,Cameron Young
1,Xander Schauffele,Tommy Fleetwood,Byeong Hun An,Collin Morikawa,Sungjae Im,Rickie Fowler,Sahith Theegala,Erik Van Rooyen
2,Jordan Spieth,Min Woo Lee,Stephan Jaeger,J.T. Poston,Russell Henley,Sam Ryder,Will Zalatoris,Akshay Bhatia
3,Jason Day,Corey Conners,Matthieu Pavon,Matthew Fitzpatrick,Chris Kirk,Cameron Champ,Emiliano Grillo,Justin Suh
4,Brian Harman,Shane Lowry,Tom Hoge,Keegan Bradley,Justin Rose,Nick Dunlap,Christiaan Bezuidenhout,Camilo Villegas
5,Harris English,Jake Knapp,Chesson Hadley,Nicolai Hojgaard,Daniel Berger,Parker Coody,Kurt Kitayama,Thorbjorn Olesen
